# Introduction to Intake-ESM

**Max Grover**

**08 December 2021**

---

# Overview

1. What is Intake-ESM?
2. How can we use it to read in data?
3. How do we work with dictionaries of datasets?
4. How do I write my analysis to work with Intake-ESM?

# Prerequisites

|Concepts | Importance | Notes |
| --- | --- | --- |
| [Introduction to Xarray](https://foundations.projectpythia.org/core/xarray/xarray.html) | Necessary | - |
| [Introduction to Matplotlib](https://foundations.projectpythia.org/core/matplotlib/matplotlib.html) | Helpful | - |

- Time to learn: 50 minutes.

---

# Setup

## Imports

In [1]:
import intake
from distributed import Client, LocalCluster
import xarray as xr
import matplotlib.pyplot as plt
import fsspec

## Spin up a Dask Cluster

In [2]:
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50770,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:50781,Total threads: 2
Dashboard: /proxy/8787/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:50773,


### Investigate the files on AWS from the CESM-LENS Dataset

In [3]:
fs = fsspec.filesystem('s3', anon=True)
bucket = 'ncar-cesm-lens'
fs.ls(bucket)

['ncar-cesm-lens/atm',
 'ncar-cesm-lens/catalogs',
 'ncar-cesm-lens/ice_nh',
 'ncar-cesm-lens/ice_sh',
 'ncar-cesm-lens/lnd',
 'ncar-cesm-lens/ocn']

In [4]:
bucket = 'ncar-cesm-lens/ocn'
fs.ls(bucket)

['ncar-cesm-lens/ocn/',
 'ncar-cesm-lens/ocn/monthly',
 'ncar-cesm-lens/ocn/static']

In [5]:
bucket = 'ncar-cesm-lens/ocn/monthly'
fs.ls(bucket)

['ncar-cesm-lens/ocn/monthly/',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-DIC.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-DOC.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-FW.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-HMXL.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-O2.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-PD.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-QFLUX.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-QSW_HBL.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-QSW_HTP.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-RESID_S.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-SALT.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-SFWF.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-SFWF_WRST.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-SHF.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-SHF_QSW.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-SSH.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-SST.zarr',
 'ncar-cesm-lens/ocn/monthly/cesmLE-20C-TAUX.zarr',
 'ncar-cesm-lens/ocn/mont

### Loading in Data Using Xarray

In [6]:
var = 'T'
atmosphere_ds = xr.open_zarr(f's3://ncar-cesm-lens/atm/monthly/cesmLE-RCP85-{var}.zarr', storage_options={'anon':True})
atmosphere_ds

<xarray.Dataset>
Dimensions:    (member_id: 40, time: 1140, lev: 30, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lev        (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 1 2 3 4 5 6 7 8 ... 34 35 101 102 103 104 105
  * time       (time) object 2006-01-16 12:00:00 ... 2100-12-16 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(1140, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    T          (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 18, 30, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.0
    NCO:                       4.3.4
    Version:                   $Name$
    host:                      tcs-f02n07
    important_note:            This data is part of the project 'Blind Evalua...
    initial_file:              b.e11.B20TRC5CNBDRD.f09_g16.105.cam.i.2006-01-...
    logname:                   mudryk
    nco_openmp_thread_number:  1
    revision_Id:               $Id$
    source:                    CAM
    title:                     UNSET
    topography_file:           /scratch/p/pjk/mudryk/cesm1_1_2_LENS/inputdata...

In [7]:
variables = ['T', 'Q']
ds_list = []

# Loop through the different files and add them to the list of datasets
for var in variables:
    ds_list.append(xr.open_zarr(f's3://ncar-cesm-lens/atm/monthly/cesmLE-RCP85-{var}.zarr', storage_options={'anon':True}))
    
atmosphere_merged = xr.merge(ds_list)
atmosphere_merged

<xarray.Dataset>
Dimensions:    (member_id: 40, time: 1140, lev: 30, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lev        (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 1 2 3 4 5 6 7 8 ... 34 35 101 102 103 104 105
  * time       (time) object 2006-01-16 12:00:00 ... 2100-12-16 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(1140, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    T          (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 18, 30, 192, 288), meta=np.ndarray>
    Q          (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 18, 30, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.0
    NCO:                       4.3.4
    Version:                   $Name$
    host:                      tcs-f02n07
    important_note:            This data is part of the project 'Blind Evalua...
    initial_file:              b.e11.B20TRC5CNBDRD.f09_g16.105.cam.i.2006-01-...
    logname:                   mudryk
    nco_openmp_thread_number:  1
    revision_Id:               $Id$
    source:                    CAM
    title:                     UNSET
    topography_file:           /scratch/p/pjk/mudryk/cesm1_1_2_LENS/inputdata...

## Using Intake-ESM
We can use the Intake-ESM catalog from the CESM-LENS dataset (`https://raw.githubusercontent.com/NCAR/cesm-lens-aws/master/intake-catalogs/aws-cesm1-le.json`) to work with these data!

In [8]:
data_catalog = intake.open_esm_datastore('https://raw.githubusercontent.com/NCAR/cesm-lens-aws/master/intake-catalogs/aws-cesm1-le.json')
data_catalog

,unique
variable,78
long_name,75
component,5
experiment,4
frequency,6
vertical_levels,3
spatial_domain,5
units,25
start_time,12
end_time,13


In [9]:
data_catalog.df

,variable,long_name,component,experiment,frequency,vertical_levels,spatial_domain,units,start_time,end_time,path
0,FLNS,net longwave flux at surface,atm,20C,daily,1.0,global,W/m2,1920-01-01 12:00:00,2005-12-31 12:00:00,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLNS....
1,FLNSC,clearsky net longwave flux at surface,atm,20C,daily,1.0,global,W/m2,1920-01-01 12:00:00,2005-12-31 12:00:00,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLNSC...
2,FLUT,upwelling longwave flux at top of model,atm,20C,daily,1.0,global,W/m2,1920-01-01 12:00:00,2005-12-31 12:00:00,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FLUT....
3,FSNS,net solar flux at surface,atm,20C,daily,1.0,global,W/m2,1920-01-01 12:00:00,2005-12-31 12:00:00,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FSNS....
4,FSNSC,clearsky net solar flux at surface,atm,20C,daily,1.0,global,W/m2,1920-01-01 12:00:00,2005-12-31 12:00:00,s3://ncar-cesm-lens/atm/daily/cesmLE-20C-FSNSC...
...,...,...,...,...,...,...,...,...,...,...,...
437,WVEL,vertical velocity,ocn,RCP85,monthly,60.0,global_ocean,centimeter/s,2006-01-16 12:00:00,2100-12-16 12:00:00,s3://ncar-cesm-lens/ocn/monthly/cesmLE-RCP85-W...
438,NaN,NaN,ocn,CTRL,static,NaN,global_ocean,NaN,NaN,NaN,s3://ncar-cesm-lens/ocn/static/grid.zarr
439,NaN,NaN,ocn,HIST,static,NaN,global_ocean,NaN,NaN,NaN,s3://ncar-cesm-lens/ocn/static/grid.zarr
440,NaN,NaN,ocn,RCP85,static,NaN,global_ocean,NaN,NaN,NaN,s3://ncar-cesm-lens/ocn/static/grid.zarr


## search

In [11]:
search1 = data_catalog.search(variable=['T','Q','TEMP','SALT'],frequency='monthly')
search1

,unique
variable,4
long_name,4
component,2
experiment,4
frequency,1
vertical_levels,2
spatial_domain,2
units,4
start_time,4
end_time,4


In [12]:
search1.df

,variable,long_name,component,experiment,frequency,vertical_levels,spatial_domain,units,start_time,end_time,path
0,Q,specific humidity,atm,20C,monthly,30.0,global,kg/kg,1920-01-16 12:00:00,2005-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-20C-Q.zarr
1,T,temperature,atm,20C,monthly,30.0,global,K,1920-01-16 12:00:00,2005-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-20C-T.zarr
2,Q,specific humidity,atm,CTRL,monthly,30.0,global,kg/kg,0400-01-16 12:00:00,2200-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-CTRL-Q....
3,T,temperature,atm,CTRL,monthly,30.0,global,K,0400-01-16 12:00:00,2200-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-CTRL-T....
4,Q,specific humidity,atm,HIST,monthly,30.0,global,kg/kg,1850-01-16 12:00:00,1919-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-HIST-Q....
5,T,temperature,atm,HIST,monthly,30.0,global,K,1850-01-16 12:00:00,1919-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-HIST-T....
6,Q,specific humidity,atm,RCP85,monthly,30.0,global,kg/kg,2006-01-16 12:00:00,2100-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-RCP85-Q...
7,T,temperature,atm,RCP85,monthly,30.0,global,K,2006-01-16 12:00:00,2100-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-RCP85-T...
8,SALT,salinity,ocn,20C,monthly,60.0,global_ocean,gram/kilogram,1920-01-16 12:00:00,2005-12-16 12:00:00,s3://ncar-cesm-lens/ocn/monthly/cesmLE-20C-SAL...
9,TEMP,potential temperature,ocn,20C,monthly,60.0,global_ocean,degC,1920-01-16 12:00:00,2005-12-16 12:00:00,s3://ncar-cesm-lens/ocn/monthly/cesmLE-20C-TEM...


## load the data

In [14]:
dsets = search1.to_dataset_dict(storage_options={'anon':True})
dsets.keys()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency'


/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: Use

dict_keys(['ocn.RCP85.monthly', 'atm.20C.monthly', 'atm.RCP85.monthly', 'atm.CTRL.monthly', 'ocn.20C.monthly', 'ocn.CTRL.monthly', 'atm.HIST.monthly', 'ocn.HIST.monthly'])

In [15]:
oceanCTL_ds = dsets['ocn.CTRL.monthly']
oceanCTL_ds

<xarray.Dataset>
Dimensions:     (member_id: 1, time: 21612, z_t: 60, nlat: 384, nlon: 320, d2: 2)
Coordinates:
  * member_id   (member_id) int64 1
  * time        (time) object 0400-01-16 12:00:00 ... 2200-12-16 12:00:00
    time_bound  (time, d2) object dask.array<chunksize=(10806, 2), meta=np.ndarray>
  * z_t         (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
Dimensions without coordinates: nlat, nlon, d2
Data variables:
    SALT        (member_id, time, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 6, 60, 384, 320), meta=np.ndarray>
    TEMP        (member_id, time, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 6, 60, 384, 320), meta=np.ndarray>
Attributes: (12/20)
    Conventions:                       CF-1.0; http://www.cgd.ucar.edu/cms/ea...
    NCO:                               4.3.4
    calendar:                          All years have exactly  365 days.
    cell_methods:                      cell_methods = time: mean ==> the vari...
    contents:                          Diagnostic and Prognostic Variables
    nco_openmp_thread_number:          1
    ...                                ...
    intake_esm_attrs:vertical_levels:  60.0
    intake_esm_attrs:spatial_domain:   global_ocean
    intake_esm_attrs:start_time:       0400-01-16 12:00:00
    intake_esm_attrs:end_time:         2200-12-16 12:00:00
    intake_esm_attrs:_data_format_:    zarr
    intake_esm_dataset_key:            ocn.CTRL.monthly

In [16]:
oceanCTL_ds.TEMP

<xarray.DataArray 'TEMP' (member_id: 1, time: 21612, z_t: 60, nlat: 384,
                          nlon: 320)>
dask.array<open_dataset-5e81bd54eb6df8a45cf897f7ded6de99TEMP, shape=(1, 21612, 60, 384, 320), dtype=float32, chunksize=(1, 6, 60, 384, 320), chunktype=numpy.ndarray>
Coordinates:
  * member_id  (member_id) int64 1
  * time       (time) object 0400-01-16 12:00:00 ... 2200-12-16 12:00:00
  * z_t        (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
Dimensions without coordinates: nlat, nlon
Attributes:
    cell_methods:  time: mean
    coordinates:   TLONG TLAT z_t time
    grid_loc:      3111
    long_name:     Potential Temperature
    units:         degC

In [19]:
result = oceanCTL_ds.TEMP.mean(dim=['z_t','nlat','nlon'])

In [21]:
%%time
result = result.compute()

/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


KeyboardInterrupt: 

Process Dask Worker process (from Nanny):
2023-02-02 16:20:53,707 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2023-02-02 16:20:53,710 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/distributed/process.py", line 188, in _run
    target(*args, **kwargs)
Traceback (most recent call last):
  File "/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/distributed/nanny.py", line 986, in _run
    asyncio.run(run())
  File "/

# The main steps

## 1. load in catalog

In [22]:
data_catalog = intake.open_esm_datastore('https://raw.githubusercontent.com/NCAR/cesm-lens-aws/master/intake-catalogs/aws-cesm1-le.json')
data_catalog

,unique
variable,78
long_name,75
component,5
experiment,4
frequency,6
vertical_levels,3
spatial_domain,5
units,25
start_time,12
end_time,13


In [25]:
catalog_subset = data_catalog.search(component='atm',
                                     variable=['T','Q'],
                                     frequency='monthly',
                                     experiment=['RCP85','20C'])
catalog_subset.df

,variable,long_name,component,experiment,frequency,vertical_levels,spatial_domain,units,start_time,end_time,path
0,Q,specific humidity,atm,20C,monthly,30.0,global,kg/kg,1920-01-16 12:00:00,2005-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-20C-Q.zarr
1,T,temperature,atm,20C,monthly,30.0,global,K,1920-01-16 12:00:00,2005-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-20C-T.zarr
2,Q,specific humidity,atm,RCP85,monthly,30.0,global,kg/kg,2006-01-16 12:00:00,2100-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-RCP85-Q...
3,T,temperature,atm,RCP85,monthly,30.0,global,K,2006-01-16 12:00:00,2100-12-16 12:00:00,s3://ncar-cesm-lens/atm/monthly/cesmLE-RCP85-T...


In [26]:
catalog_subset.df.variable.unique()

array(['Q', 'T'], dtype=object)

In [28]:
dsets2 = catalog_subset.to_dataset_dict(storage_options={'anon':True})
dsets2.keys()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency'


/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


dict_keys(['atm.RCP85.monthly', 'atm.20C.monthly'])

In [29]:
historical_ds = dsets2['atm.20C.monthly']
future_ds = dsets2['atm.RCP85.monthly']

In [30]:
lat = 40.015
lon = 105.2705
variable = 'T'

historical_ds.isel(member_id=0,lev=-1).sel(lat=lat,lon=lon,method='nearest')

<xarray.Dataset>
Dimensions:    (time: 1032, nbnd: 2)
Coordinates:
    lat        float64 40.05
    lev        float64 992.6
    lon        float64 105.0
    member_id  int64 1
  * time       (time) object 1920-01-16 12:00:00 ... 2005-12-16 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(1032, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    Q          (time) float32 dask.array<chunksize=(18,), meta=np.ndarray>
    T          (time) float32 dask.array<chunksize=(18,), meta=np.ndarray>
Attributes: (12/20)
    Conventions:                       CF-1.0
    NCO:                               4.3.4
    Version:                           $Name$
    important_note:                    This data is part of the project 'Blin...
    initial_file:                      b.e11.B20TRC5CNBDRD.f09_g16.001.cam.i....
    logname:                           mudryk
    ...                                ...
    intake_esm_attrs:vertical_levels:  30.0
    intake_esm_attrs:spatial_domain:   global
    intake_esm_attrs:start_time:       1920-01-16 12:00:00
    intake_esm_attrs:end_time:         2005-12-16 12:00:00
    intake_esm_attrs:_data_format_:    zarr
    intake_esm_dataset_key:            atm.20C.monthly

In [35]:
%%time
output = historical_ds.isel(member_id=0,lev=-1).sel(lat=lat,lon=lon,method='nearest').isel(time=range(2)).T.compute()

/Users/moo270/miniconda3/envs/pangeo_mac/lib/python3.10/site-packages/dask/base.py:1373: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


KeyboardInterrupt: 

In [36]:
client.shutdown()